In [38]:
#!pip install gensim==3.8 pymorphy2 razdel

In [1]:
import zipfile

import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd

import gensim 
import pymorphy2
import razdel

import sklearn
import sklearn.linear_model
import sklearn.model_selection
import sklearn.preprocessing

from tqdm.auto import tqdm
tqdm.pandas()

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xeniyapchelkina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#! wget -O data.csv https://www.dropbox.com/s/96rv8lcpbog4chl/data.csv?dl=0

Вам предоставляется набор из 2156 текстов новостных статей по различной тематике, собранных из интернета. Данные представлены в файле data.csv в виде текст-тема. Вам необходимо проанализировать данные тексты и построить модель, которая будет предсказывать тему новости.
В ReadMe необходимо обосновать выбор модели и метрик по ее оценке.


In [6]:
df = pd.read_csv('data.csv')
df.shape

(2156, 2)

In [7]:
df.sample(n=5)

,text,label
9,CGTN: Китай призывает к многостороннему подход...,Международные отношения
1192,Названо лучшее время для покупки долларов\nМос...,не по теме
687,Дагестан перешёл на самоочищение\nНейтрализова...,не по теме
487,ВСУ обстреляли окраины Горловки из гранатомёта...,военная тематика
1667,Россиянин украл яхту в Турции и сел на мель в ...,военная тематика


In [9]:
df['text'].apply(len).describe(percentiles=[0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1.0])

count     2156.000000
mean      6459.500464
std       9858.465399
min        339.000000
50%       1998.500000
60%       2617.000000
70%       3959.500000
80%       6690.000000
90%      27676.000000
95%      31083.000000
100%     41822.000000
max      41822.000000
Name: text, dtype: float64

In [12]:
df['label'].value_counts()  # spam / ne-spam

не по теме                 891
политика                   357
COVID-19                   249
Международные отношения    177
мнения                     131
военная тематика           112
аналитика                  101
торговля                    72
меры поддержки              31
проекты                     12
Россия                      10
инвестиция                   9
Социологические опросы       4
Name: label, dtype: int64

In [13]:
df['text'].apply(lambda x: x.count('\n')).describe()

count    2156.0
mean        2.0
std         0.0
min         2.0
25%         2.0
50%         2.0
75%         2.0
max         2.0
Name: text, dtype: float64

In [14]:
print(df['text'].sample().item())

Пентагон незаконно шпионит за гражданами на территории США
В Сенате уже высказывают опасения Пентагон вновь оказался в центре скандала. Издание New York Times сообщило, что ведомственная разведка уже несколько лет покупает базы данных с информацией о перемещениях американцев и иностранных граждан. По закону США, для получения этих сведений властям сначала нужно запрашивать ордер, чтобы на его основании требовать от телефонных операторов данные геолокации своих клиентов. Однако Пентагон нашёл лазейку, покупая аналогичную базу у посредников без решения суда. Авторы статьи предположили, что информация используется главным образом для расследований об иностранцах за границей. В Сенате такую практику назвали посягательством на права граждан, сообщает "ТВ Центр". Все самое интересное - в нашем канале "Яндекс.Дзен"
https://www.tvc.ru/news/show/id/202339


In [15]:
df['is_spam'] = df['label'].apply(lambda x: 1 if x == 'не по теме' else 0)
df['is_spam'].value_counts()

0    1265
1     891
Name: is_spam, dtype: int64

In [16]:
# для быстроты обойдусь без валидационного сета
train_df, test_df = sklearn.model_selection.train_test_split(df, test_size=0.2, random_state=23, shuffle=True, stratify=df['label'])
assert set(train_df['label']) == set(test_df['label'])
train_df.shape, test_df.shape

((1724, 3), (432, 3))

In [17]:
def custom_bag_of_words(text):
    """
    пытаюст услилить полезный сигнал, убрать шумы, сделать пригодным для подачи в модель эмбеддингов
    """
    
    splitted = text.split('\n')
    assert len(splitted) == 3
    (title, body, link) = splitted

    max_title_chars = 150
    max_body_chars = 150 * 7
    title = ' '.join([x for x in title[:max_title_chars].split(' ')][:-1])
    body = ' '.join([x for x in body[:max_body_chars].split(' ')][:-1])

    russian_stopwords = nltk.corpus.stopwords.words('russian')
    wanted_pos = [
        'NOUN',  # 	имя существительное
        'ADJF',  # 	имя прилагательное (полное)
        'VERB',  # 	глагол (личная форма)
        'INFN',  # 	глагол (инфинитив)
    ]
    
    tokenized_title = [x.text for x in razdel.tokenize(title)]
    tokenized_title = [morph.parse(x)[0].normal_form for x in tokenized_title if morph.parse(x)[0].tag.POS in wanted_pos]
    tokenized_title = [x for x in tokenized_title if x not in russian_stopwords]
    
    tokenized_body = [x.text for x in razdel.tokenize(body)]
    tokenized_body = [morph.parse(x)[0].normal_form for x in tokenized_body if morph.parse(x)[0].tag.POS in wanted_pos]
    tokenized_body = [x for x in tokenized_body if x not in russian_stopwords]

    title_stronger_signal_coeff = 2
    bag_of_words = title_stronger_signal_coeff * tokenized_title + tokenized_body
    return ' '.join(bag_of_words)

In [18]:
train_df['bag_of_words'] = train_df['text'].progress_apply(custom_bag_of_words)
test_df['bag_of_words'] = test_df['text'].progress_apply(custom_bag_of_words)

  0%|          | 0/1724 [00:00<?, ?it/s]

<ipython-input-18-9d510ab4f8e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['bag_of_words'] = train_df['text'].progress_apply(custom_bag_of_words)


  0%|          | 0/432 [00:00<?, ?it/s]

<ipython-input-18-9d510ab4f8e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['bag_of_words'] = test_df['text'].progress_apply(custom_bag_of_words)


In [19]:
train_df['bag_of_words'].apply(len).describe()

count    1724.000000
mean      813.662993
std       125.325010
min       257.000000
25%       761.000000
50%       839.000000
75%       899.000000
max      1068.000000
Name: bag_of_words, dtype: float64

In [ ]:
! wget http://vectors.nlpl.eu/repository/20/187.zip

In [ ]:
with zipfile.ZipFile('187.zip', 'r') as zip_ref:
    zip_ref.extractall('w2v_model/')

In [22]:
word2vec_model = gensim.models.KeyedVectors.load('187/model.model')
#word2vec_model.init_sims(replace=True)  # на случай cos-sim

In [23]:
def l2_norm(x):
   return np.sqrt(np.sum(x**2))

def div_norm(x):
   norm_value = l2_norm(x)
   if norm_value > 0:
       return x * ( 1.0 / norm_value)
   else:
       return x

def emb_single(word, model):
    if word == '':
        return np.zeros(model.vector_size)
    return model.get_vector(word)

def emb(bag_of_words, model):
    words = bag_of_words.split(' ')
    embds = [emb_single(word=x, model=model) for x in words]
    normed = [div_norm(x) for x in embds]
    avg = np.mean(np.array(normed), axis=0)
    return avg

## Спам / не спам

In [24]:
X_train_spam = train_df['bag_of_words'].progress_apply(lambda x: emb(bag_of_words=x, model=word2vec_model))
X_train_spam = np.array(X_train_spam.values.tolist())
assert X_train_spam.shape == (len(train_df), word2vec_model.vector_size)

y_train_spam = train_df['is_spam'].values

#####


X_test_spam= test_df['bag_of_words'].progress_apply(lambda x: emb(bag_of_words=x, model=word2vec_model))
X_test_spam = np.array(X_test_spam.values.tolist())
assert X_test_spam.shape == (len(test_df), word2vec_model.vector_size)

y_test_spam = test_df['is_spam'].values

  0%|          | 0/1724 [00:00<?, ?it/s]

  0%|          | 0/432 [00:00<?, ?it/s]

In [25]:
clf_is_spam = sklearn.linear_model.LogisticRegression(
    class_weight='balanced',
    random_state=23,
)

clf_is_spam.fit(X_train_spam, y_train_spam)

y_pred_spam = clf_is_spam.predict(X_test_spam)

print(sklearn.metrics.classification_report(y_true=y_test_spam, y_pred=y_pred_spam, target_names=['not spam', 'spam']))
print('F1 score:', sklearn.metrics.f1_score(y_true=y_test_spam, y_pred=y_pred_spam))

              precision    recall  f1-score   support

    not spam       0.85      0.85      0.85       253
        spam       0.78      0.79      0.79       179

    accuracy                           0.82       432
   macro avg       0.82      0.82      0.82       432
weighted avg       0.82      0.82      0.82       432

F1 score: 0.7855153203342619


In [26]:
test_df['pred_is_spam'] = y_pred_spam

<ipython-input-26-b6dcc6ff46a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['pred_is_spam'] = y_pred_spam


## Далее уже классификатор по темам

In [27]:
notspam_train_df = train_df[train_df['is_spam'] != 1]
notspam_test_df = test_df[test_df['pred_is_spam'] != 1]  # тут мы внесем ошибки, но мы согласны такой пайплайн. в метрике отдельно будем считать пропагейт ошибок

len(notspam_train_df), len(notspam_test_df)

(1012, 252)

In [29]:
notspam_test_df = notspam_test_df[notspam_test_df['label'] != 'не по теме']  # не совсем честно, но в метркие можно учесть

In [30]:
set(notspam_train_df['label'].unique()) == set(notspam_test_df['label'].unique())

True

In [31]:
label_encoder = sklearn.preprocessing.LabelEncoder()
# сделаю на всех данных, чтобы не прошляпить класс "не по теме", потому что из обучения выкину
label_encoder.fit(notspam_train_df['label'])
print(label_encoder.classes_)

notspam_train_df['target'] = label_encoder.transform(notspam_train_df['label'])
notspam_test_df['target'] = label_encoder.transform(notspam_test_df['label'])

['COVID-19' 'Международные отношения' 'Россия' 'Социологические опросы'
 'аналитика' 'военная тематика' 'инвестиция' 'меры поддержки' 'мнения'
 'политика' 'проекты' 'торговля']


<ipython-input-31-de34eba12ae7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notspam_train_df['target'] = label_encoder.transform(notspam_train_df['label'])


In [32]:
X_train = notspam_train_df['bag_of_words'].progress_apply(lambda x: emb(bag_of_words=x, model=word2vec_model))
X_train = np.array(X_train.values.tolist())
assert X_train.shape == (len(notspam_train_df), word2vec_model.vector_size)

y_train = notspam_train_df['target'].values


#####

X_test = notspam_test_df['bag_of_words'].progress_apply(lambda x: emb(bag_of_words=x, model=word2vec_model))
X_test = np.array(X_test.values.tolist())
assert X_test.shape == (len(notspam_test_df), word2vec_model.vector_size)

y_test = notspam_test_df['target'].values

  0%|          | 0/1012 [00:00<?, ?it/s]

  0%|          | 0/214 [00:00<?, ?it/s]

In [34]:
clf = sklearn.linear_model.LogisticRegression(
    class_weight='balanced',
    random_state=23,
    multi_class='multinomial',
)

clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', multi_class='multinomial',
                   random_state=23)

In [35]:
y_pred = clf.predict(X_test)  # predict_proba
#assert y_pred.shape == (len(X_test), len(label_encoder.classes_)) # if predict_proba

In [36]:
sklearn.metrics.f1_score(y_true=y_test, y_pred=y_pred, average='micro')#

0.5186915887850467

In [37]:
print(sklearn.metrics.classification_report(y_true=y_test.tolist(), y_pred=y_pred))#, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

           0       0.91      0.72      0.81        40
           1       1.00      0.03      0.06        33
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.45      0.29      0.36        17
           5       0.64      0.84      0.73        19
           6       0.10      0.50      0.17         2
           7       0.25      0.50      0.33         4
           8       0.33      0.15      0.21        20
           9       0.64      0.72      0.68        67
          10       0.20      1.00      0.33         1
          11       0.50      0.56      0.53         9

    accuracy                           0.52       214
   macro avg       0.42      0.44      0.35       214
weighted avg       0.68      0.52      0.52       214



In [ ]:
#         precision    recall  f1-score   support

#            0       0.00      0.00      0.00        50
#            1       0.00      0.00      0.00        36
#            2       0.00      0.00      0.00         2
#            3       0.00      0.00      0.00         1
#            4       0.00      0.00      0.00        20
#            5       0.00      0.00      0.00        22
#            6       0.00      0.00      0.00         2
#            7       0.00      0.00      0.00         6
#            8       0.00      0.00      0.00        26
#            9       0.41      1.00      0.59       179
#           10       0.00      0.00      0.00        72
#           11       0.00      0.00      0.00         2
#           12       0.00      0.00      0.00        14

#     accuracy                           0.41       432
#    macro avg       0.03      0.08      0.05       432
# weighted avg       0.17      0.41      0.24       432
#
# Без разделения на спам/не-спам было много нулей в этой конфьюжн-матрице, кроме строки класса "не по теме"
# Видимо в этом была проблема.
# Сейчас её нет, но теперр проблема что нечестно в одном месте опирались на не-спам в тесте, хотя дальше работали с той частью которую сами запредиктили.
# По-хорошему надо по-другом узамерить пропагейт ошибки с первой модели.